In [ ]:
pip install openai azure-storage-blob azure-identity tenacity


In [ ]:
pip install -r requirements.txt

In [10]:
import os
import tiktoken
import openai
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from tenacity import retry, wait_random_exponential, stop_after_attempt

# Load environment variables
load_dotenv()

# Option 1 - Use Azure AD authentication with az cli (use az login in terminal)
# default_credential = DefaultAzureCredential()
# token = default_credential.get_token("https://cognitiveservices.azure.com/.default")
# openai.api_type = "azure_ad"
# openai.api_base = os.environ.get("OPENAI_API_BASE")
# openai.api_key = token.token
# openai.api_version = "2022-12-01"


azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# Define embedding model and encoding
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
encoding = tiktoken.get_encoding('cl100k_base')


Test if completitions work:

In [27]:
import os
from openai import AzureOpenAI


client = AzureOpenAI(
  azure_endpoint = azure_endpoint, 
  api_key=api_key,
  api_version="2024-02-01"
)

model='gpt-4o-mini'
response = client.chat.completions.create(
model=model,
messages=[
    {"role": "system", "content": "YOU ARE USEFUL ASSISTANT"},
    {"role": "user", "content": "How are you"}
    ])

print(response.choices[0].message.content)



I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?


Do it in a streaming fashion:

Test if embeddings work:

In [28]:
response = client.embeddings.create(
    input = "Your text string goes here",
    model= "text-embedding-ada-002"  # model = "deployment_name".
)

print(response.model_dump_json(indent=2))

{
  "data": [
    {
      "embedding": [
        -0.007578954566270113,
        -0.0055061643943190575,
        0.011402026750147343,
        -0.0247525442391634,
        -0.024873483926057816,
        0.039802949875593185,
        -0.010347154922783375,
        -0.009480412118136883,
        -0.013384111225605011,
        -0.00991042423993349,
        -0.011717816814780235,
        0.008217253722250462,
        -0.014338199980556965,
        0.0077603659592568874,
        0.010044802911579609,
        -0.0049820877611637115,
        0.022777177393436432,
        -0.0017082883277907968,
        0.015319163911044598,
        -0.010279965586960316,
        0.0049182577058672905,
        0.012208298780024052,
        0.004834271036088467,
        0.010568879544734955,
        -0.006574474740773439,
        -0.00044596908264793456,
        0.005707732401788235,
        -0.012853316031396389,
        0.01634044200181961,
        0.004454651847481728,
        0.0063762660138309,
        -0.0

Test if tokenizer works:

In [29]:
tokens = encoding.encode("Hola como estamos que tal todo")
print(tokens)
print(len(tokens))

[69112, 8112, 68160, 1744, 8374, 12067]
6


# Ejercicio 1: Generación de texto con diferentes prompts
# Objetivo: Experimentar con distintos prompts para observar cómo varía la respuesta del modelo.


In [30]:
prompts = [
    "Escribe un poema sobre el mar",
    "Resume los últimos avances en IA",
    "Explica la física cuántica en términos simples",
    "Genera una lista de ideas creativas para startups"
]


for prompt in prompts:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Eres un asistente sabio, escribe una respuesta corta."},
            {"role": "user", "content": prompt}
        ]
    )
    print("----------------------")
    print(f"Prompt: {prompt}")
    print(f"Respuesta: {response.choices[0].message.content}\n")
    print("----------------------")


----------------------
Prompt: Escribe un poema sobre el mar
Respuesta: En el vasto abrazo del mar profundo,  
susurra el viento secretos del mundo.  
Las olas danzan con suave compás,  
bailarinas de espuma, un eterno jamás.  

Los rayos del sol se deslizan, dorados,  
pintando la superficie en sueños alados.  
En sus aguas brillantes, reflejos de paz,  
el horizonte promete un nuevo compás.  

Las gaviotas cantan, su vuelo es libertad,  
y el eco del océano, un canto de eternidad.  
Así, el mar guarda historias sin fin,  
misterios navegantes, un eterno festín.  

----------------------
----------------------
Prompt: Resume los últimos avances en IA
Respuesta: Hasta octubre de 2023, algunos de los últimos avances en inteligencia artificial incluyen el desarrollo de modelos de lenguaje más potentes y eficientes, como las versiones mejoradas de GPT y otras IA generativas que permiten la creación de contenido de alta calidad. También se han realizado progresos en la IA aplicada a la sal

# Ejercicio 2: Comparación de similitud entre textos usando embeddings
# Objetivo: Calcular la similitud coseno entre dos fragmentos de texto.

In [31]:
from sklearn.metrics.pairwise import cosine_similarity


In [34]:
text1 = "El gato de mi prima es feo."
text2 = "La inteligencia artificial impulsa el aprendizaje automático."
text3 = "El arroz con pollo es riquisimo."


embedding1 = client.embeddings.create(input=text1, model=EMBEDDING_MODEL).data[0].embedding
embedding2 = client.embeddings.create(input=text2, model=EMBEDDING_MODEL).data[0].embedding
embedding3 = client.embeddings.create(input=text3, model=EMBEDDING_MODEL).data[0].embedding


import numpy as np  
from sklearn.metrics.pairwise import cosine_similarity  

embedding1 = np.array(embedding1).reshape(1, -1)  
embedding2 = np.array(embedding2).reshape(1, -1)  
embedding3 = np.array(embedding3).reshape(1, -1)  

similarity_1_2 = cosine_similarity(embedding1, embedding2)[0][0]  
similarity_1_3 = cosine_similarity(embedding1, embedding3)[0][0]  

print(f"Similitud entre text1 y text2: {similarity_1_2:.2f}")  
print(f"Similitud entre text1 y text3: {similarity_1_3:.2f}")  


Similitud entre text1 y text2: 0.74
Similitud entre text1 y text3: 0.79


# Ejercicio 3: Tokenización de frases complejas
# Objetivo: Practicar la tokenización de frases más largas y analizar la cantidad de tokens generados.

In [36]:
sentences = [
    "El sol brilla intensamente sobre las montañas mientras el viento suavemente acaricia los árboles.",
    "Las estrellas en el cielo nocturno parecen contar historias de tiempos antiguos",
    "La inteligencia artificial está revolucionando el mundo al automatizar tareas complejas y mejorar la eficiencia en diversas industrias."
]

for sentence in sentences:
    tokens = encoding.encode(sentence)
    print(f"Oración: {sentence}")
    print(f"Tokens: {tokens}")
    print(f"Número de tokens: {len(tokens)}\n")

Oración: El sol brilla intensamente sobre las montañas mientras el viento suavemente acaricia los árboles.
Tokens: [6719, 2092, 1437, 6374, 25228, 9157, 15482, 5252, 20605, 33469, 300, 59694, 658, 348, 11110, 924, 402, 39961, 1645, 277, 24985, 2537, 19216, 81, 754, 645, 13]
Número de tokens: 27

Oración: Las estrellas en el cielo nocturno parecen contar historias de tiempos antiguos
Tokens: [44903, 1826, 74023, 665, 658, 12088, 20782, 93101, 399, 78, 24068, 48422, 89805, 7135, 3557, 409, 9165, 336, 981, 3276, 27843, 437]
Número de tokens: 22

Oración: La inteligencia artificial está revolucionando el mundo al automatizar tareas complejas y mejorar la eficiencia en diversas industrias.
Tokens: [8921, 90745, 8968, 21075, 15833, 5891, 92660, 4988, 658, 29452, 453, 61852, 11403, 259, 33637, 1946, 30826, 379, 85216, 1208, 31469, 3457, 8968, 665, 99383, 9047, 48931, 13]
Número de tokens: 28



# Ejercicio 4: Generación de texto en streaming
# Objetivo: Implementar una generación de texto que muestre la respuesta en tiempo real (streaming).

In [37]:
import time

stream_response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "Eres un asistente de narración de historias."},
        {"role": "user", "content": "Cuéntame una historia sobre un valiente caballero."}
    ],
    stream=True
)

for chunk in stream_response:
    if chunk.choices:
        print(chunk.choices[0].delta.content or "", end="", flush=True)
        time.sleep(0.05)

print("\n\n--- Fin de la Historia ---")

Había una vez, en un reino lejano conocido como Eldoria, un valiente caballero llamado Sir Cedric. Era conocido por su valentía y nobleza, así como por su armadura reluciente que reflejaba la luz del sol como si fuera un faro en la oscuridad. Su espada, Forjadora de Destinos, había sido un regalo del rey y era tan legendaria como el mismo Cedric.

El pueblo vivía en paz bajo el reinado de la buena reina Eliana, hasta que un día, un dragón oscuro apareció en las montañas que rodeaban Eldoria. Era un ser temido y despiadado, que aterrorizaba a las aldeas cercanas, robando ganado y destruyendo cosechas con su fuego abrasador. El rey, en su desesperación, convocó a sus más valientes guerreros, pero todos temían enfrentarse a la bestia.

Sir Cedric, sin embargo, se ofreció como voluntario. Consciente del peligro, se preparó para la batalla, sabiendo que su deber era proteger a su gente. Antes de partir, la reina le dio un amuleto especial, un talismán que había pertenecido a su madre, dicié

## Ejercicio 2: Comparación de similitudes semánticas entre dos párrafos

En este ejercicio, aprenderás a usar los embeddings de OpenAI para calcular la similitud semántica entre dos párrafos, lo que te permitirá comparar la cercanía entre textos utilizando la métrica de similitud coseno.

Pasos:

Usa el modelo de embeddings text-embedding-ada-002 de OpenAI.
Calcular la similitud coseno entre dos fragmentos de texto.
Imprimir el valor de similitud entre los párrafos.

Resultado esperado:

Similitud entre text1 y text2: 0.94
Similitud entre text1 y text3: 0.15
Explicación: Los estudiantes aprenderán cómo usar embeddings para evaluar la similitud semántica entre diferentes fragmentos de texto. En este caso, text1 y text2 son similares en contenido relacionado con la inteligencia artificial, por lo que la similitud es alta (0.94), mientras que text3 habla sobre actividades al aire libre, lo que genera una similitud baja con los otros textos.



In [38]:
# Frases de ejemplo
text1 = "Artificial intelligence is transforming the world in every sector."  # English
text2 = "La inteligencia artificial está revolucionando las industrias globalmente, cambiando la forma en que trabajamos."  # Spanish
text3 = "L'intelligence artificielle transforme le monde dans tous les secteurs."  # French

embedding1 = client.embeddings.create(input=text1, model=EMBEDDING_MODEL).data[0].embedding
embedding2 = client.embeddings.create(input=text2, model=EMBEDDING_MODEL).data[0].embedding
embedding3 = client.embeddings.create(input=text3, model=EMBEDDING_MODEL).data[0].embedding


# Calcular la similitud coseno
similarity_1_2 = cosine_similarity([embedding1], [embedding2])[0][0]
similarity_1_3 = cosine_similarity([embedding1], [embedding3])[0][0]

print(f"Similitud entre text1 y text2: {similarity_1_2:.2f}")
print(f"Similitud entre text1 y text3: {similarity_1_3:.2f}")


Similitud entre text1 y text2: 0.89
Similitud entre text1 y text3: 0.92


## Ejercicio 3: Extracción de palabras clave de un texto utilizando embeddings

Este ejercicio tiene como objetivo enseñar cómo usar OpenAI para extraer palabras clave relevantes de un texto, basándose en la similitud de los embeddings.

Pasos:

Usa un fragmento de texto.
Genera embeddings para cada palabra.
Evalúa la similitud de cada palabra con el texto completo para identificar las palabras más relevantes.

Resultado esperado:

Palabra: artificial - Similitud: 0.92
Palabra: intelligence - Similitud: 0.88
Palabra: healthcare - Similitud: 0.83
Palabra: finance - Similitud: 0.80
Palabra: education - Similitud: 0.78
Palabra: revolutionize - Similitud: 0.70

Explicación: El modelo de OpenAI se utiliza para evaluar la similitud de cada palabra clave con el contexto general del texto. Las palabras más relevantes, como "artificial" e "intelligence", tienen una mayor similitud con el texto completo, mientras que otras, como "revolutionize", tienen una menor similitud.


In [39]:


# Fragmento de texto
text = "La inteligencia artificial tiene el potencial de revolucionar múltiples sectores, incluyendo la salud, las finanzas y la educación."

# Generar embeddings para el texto completo
text_embedding = client.embeddings.create(input=text, model=EMBEDDING_MODEL).data[0].embedding

# Listado de palabras clave a evaluar
keywords = ["artificial", "inteligencia", "salud", "revolucionar", "finanzas", "educación"]

# Calcular embeddings para cada palabra y compararlos con el texto
word_similarities = {}
for word in keywords:
    word_embedding = client.embeddings.create(input=word, model=EMBEDDING_MODEL).data[0].embedding
    similarity = cosine_similarity([text_embedding], [word_embedding])[0][0]
    word_similarities[word] = similarity

# Mostrar palabras clave ordenadas por relevancia
sorted_keywords = sorted(word_similarities.items(), key=lambda x: x[1], reverse=True)

for word, similarity in sorted_keywords:
    print(f"Palabra: {word} - Similitud: {similarity:.2f}")


Palabra: inteligencia - Similitud: 0.84
Palabra: finanzas - Similitud: 0.81
Palabra: educación - Similitud: 0.81
Palabra: revolucionar - Similitud: 0.79
Palabra: salud - Similitud: 0.78
Palabra: artificial - Similitud: 0.75


## Ejercicio 4: Generación de resúmenes con OpenAI
Enunciado:
En este ejercicio, el objetivo es generar un resumen breve de un texto largo utilizando el modelo de OpenAI.

Pasos:

Usa un texto largo y genera un resumen de este utilizando un prompt adecuado.
Configura correctamente la solicitud a OpenAI para obtener un resumen conciso.

Resultado esperado:

Resumen generado:
Artificial intelligence (AI) is the intelligence demonstrated by machines, contrasting with natural intelligence. AI is defined as the study of intelligent agents, which are devices that perceive their environment and take actions to maximize their goals. The field of AI aims to mimic human cognitive functions, such as learning and problem-solving, though tasks considered to require intelligence are often excluded from AI as machines improve.

Explicación: Este ejercicio enseña cómo utilizar OpenAI para generar resúmenes de textos largos. El modelo utiliza el contexto dado para condensar la información sin perder el sentido de lo que se quiere transmitir.


In [40]:


# Texto largo
long_text = """
La inteligencia artificial (IA) es la inteligencia demostrada por las máquinas, en contraste con la inteligencia natural mostrada por los humanos y los animales. Los principales libros de texto sobre IA definen el campo como el estudio de los "agentes inteligentes": cualquier dispositivo que percibe su entorno y toma acciones que maximizan sus posibilidades de lograr sus objetivos con éxito. Coloquialmente, el término "inteligencia artificial" se usa a menudo para describir máquinas (o computadoras) que imitan las funciones "cognitivas" que los humanos asocian con la mente humana, como el "aprendizaje" y la "resolución de problemas". A medida que las máquinas se vuelven cada vez más capaces, las tareas consideradas como que requieren "inteligencia" a menudo se eliminan de la definición de IA, un fenómeno conocido como el efecto de la IA. La investigación sobre IA se ha definido como el estudio de los "agentes inteligentes".
"""

prompt = f"Proporciona un breve resumen del siguiente texto:\n{long_text}"
# Solicitar resumen

response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Eres un asistente conocedor."},
            {"role": "user", "content": prompt}
        ],
    max_tokens=100
    )

# Mostrar resumen
print(f"Resumen generado:\n{response.choices[0].message.content}")


Resumen generado:
La inteligencia artificial (IA) se refiere a la capacidad de las máquinas para exhibir inteligencia, en contraste con la inteligencia natural de humanos y animales. Se define como el estudio de "agentes inteligentes", que son dispositivos que perciben su entorno y actúan para alcanzar sus objetivos. El término también se usa para describir máquinas que imitan funciones cognitivas humanas, como el aprendizaje y la resolución de problemas. A medida que las máquinas avanzan, las tareas antes consideradas inteligentes a menudo se excl


## Ejercicio 5: Generación de respuestas basadas en conocimiento específico
Enunciado:
Este ejercicio te ayudará a generar respuestas detalladas y útiles a partir de un contexto o conocimiento específico, por ejemplo, sobre la historia de la informática.

Pasos:

Crea un conjunto de datos que contenga información básica sobre un tema.
Genera una respuesta a una pregunta específica utilizando ese contexto como referencia.

Resultado esperado:

Charles Babbage is known as the father of computing.

Explicación: Este ejercicio demuestra cómo los modelos de OpenAI pueden generar respuestas útiles basadas en información específica proporcionada en un contexto. Los estudiantes aprenderán cómo usar esta capacidad para obtener respuestas precisas dentro de un marco de conocimiento.


In [41]:

# Contexto sobre la historia de la informática
context = """
La historia de la informática se remonta al principio del siglo XIX, con figuras clave como Charles Babbage, quien es considerado el "padre de la computación" por su invención de la Máquina Analítica. A mediados del siglo XX, las computadoras se hicieron más prácticas, con el desarrollo de las primeras computadoras programables y la invención del transistor. Más tarde, el desarrollo de Internet revolucionó el campo y dio lugar al surgimiento de las tecnologías de computación modernas.
"""

# Pregunta
question = "¿Quién es conocido como el padre de la computación?"

# Solicitar respuesta basada en el contexto

response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Eres un asistente conocedor."},
            {"role": "user", "content": f"Basado en la siguiente información, responde a la pregunta:\n{context}\n\nPregunta: {question}\nRespuesta:"}
        ],
    max_tokens=100
    )

# Mostrar respuesta
print(f"Respuesta generada:\n{response.choices[0].message.content}")


Respuesta generada:
Charles Babbage es conocido como el padre de la computación.



## Ejercicio 6: Análisis de emociones en un texto

En este ejercicio, el objetivo es enseñar cómo realizar un análisis básico de emociones en un texto utilizando los modelos de OpenAI. El modelo debería identificar la emoción principal expresada en un texto.

Pasos:

Escribe varios textos que expresen diferentes emociones.
Usa OpenAI para clasificar la emoción principal de cada texto.

Resultado esperado:

Texto: I am so happy! Today was the best day of my life.
Emoción: Happiness

Texto: I feel really sad and upset after hearing the news.
Emoción: Sadness

Texto: The project is going great, I'm feeling confident and motivated.
Emoción: Confidence

Explicación: Los estudiantes aprenderán a utilizar OpenAI para realizar análisis emocionales de textos y comprender cómo los modelos pueden identificar emociones basadas en el contenido proporcionado.


In [45]:

# Textos de ejemplo con emociones
texts = [
    "¡Estoy tan feliz! Hoy fue el mejor día de mi vida.",
    "Me siento realmente triste y desconsolado después de escuchar las noticias.",
    "El proyecto va de maravilla, me siento confiado y motivado.",
]


# Solicitar análisis de emociones
for text in texts:
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "Eres un asistente conocedor que analiza sentimientos."},
            {"role": "user", "content": f"Analiza la emoción del siguiente texto:\n{text}"}
        ],
    max_tokens=10
    )
    print(f"Texto: {text}\nEmoción: {response.choices[0].message.content.strip()}\n")


Texto: ¡Estoy tan feliz! Hoy fue el mejor día de mi vida.
Emoción: El texto expresa una fuerte emoción de felicidad. El

Texto: Me siento realmente triste y desconsolado después de escuchar las noticias.
Emoción: El texto expresa una profunda tristeza y desconsuelo

Texto: El proyecto va de maravilla, me siento confiado y motivado.
Emoción: El texto expresa sentimientos positivos y optimistas. Las

